In [ ]:
# Import Libraries
import numpy as np

from project_utils.bb_model         import BB_Model
from project_utils.feature_stats    import Feature_Statistics_C
from project_utils.feature_stats    import Regression_Container
from project_utils.acq_data_capture import Acq_Data_nD
from project_utils.surroagte_gp     import Surrogate_GP



In [ ]:
Num_Points    = 5
Title         = 'Diabetes'
FUR_Weight    = 0.1
N_consistancy = 25
consist_bound = 0.3

In [ ]:
if isinstance(FUR_Weight, float):
    FUR_Weight = [FUR_Weight, 1.0 - FUR_Weight]
    
print(FUR_Weight)

In [ ]:
Title = Title + ' W:'+ str(FUR_Weight)

In [ ]:
# Load Forrester model

Class_data = BB_Model(dataset = 'Diabetes')
                      
#Class_data.df().head()

In [ ]:
Class_data.MPL()

In [ ]:
Class_data.Random_Forest()

In [ ]:
#Class_data.GP()

In [ ]:
Class_data.L_Regression()

In [ ]:
print(Class_data.get_features())

In [ ]:
#Default settings
from unravel_2.ur_model import UR_Model

X_train, X_test, y_train, y_test = Class_data.get_TT_data()

#Initializing the explainer object
URM_Exp = UR_Model(bbox_model           = Class_data.get_Random_Forest(),
                   train_data           = X_train,
                   feature_names        = Class_data.get_features(),
                   categorical_features = Class_data.get_catagorical(),
                   mode                 = Class_data.get_mode(),
                   sampling_optimize    = "opt")

#sur_GP = Surrogate_GP()


In [ ]:
#I2nd explainer object
URM_Exp_2 = UR_Model(bbox_model           = Class_data.get_L_Regression(),
                     train_data           = X_train,
                     feature_names        = Class_data.get_features(),
                     categorical_features = Class_data.get_catagorical(),
                     mode                 = Class_data.get_mode(),
                     sampling_optimize    = "opt")



In [ ]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
print('Mean 0:', mean)
print('Std 0: ', std)


In [ ]:
# Generate explantion data for Random Forest

Class_Stats   = Feature_Statistics_C(feature_names = Class_data.get_features(),
                                     mode          = Class_data.get_mode(),
                                     classes       = ['Not-Diabetic', 'Diabetic'],
                                     X_train_std   = Class_data.get_X_train_std(),
                                     N_consistancy = N_consistancy)

Class_Stats_2 = Feature_Statistics_C(feature_names = Class_data.get_features(),
                                     mode          = Class_data.get_mode(),
                                     classes       = ['Not-Diabetic', 'Diabetic'],
                                     X_train_std   = Class_data.get_X_train_std())


if Num_Points > X_test.shape[0]: Num_Points = X_test.shape[0]

elif Num_Points == 0:            Num_Points = X_test.shape[0]

for test_instance in range(Num_Points):

    print('INSTANCE: ', test_instance)
       
    data_row  = np.array([X_test[test_instance]])
    
#    data_row  = np.array([0.4])

#    print('data row ',data_row)
    
    URM_Exp.explain(X_init      = data_row.reshape(1, -1),
                    Dimension   = 'Multi D',
                    kernel_type = "Matern_3",
                    max_iter    = 20,
                    alpha       = "FUR_W",
                    #jitter=5,
                    interval    = 1,
                    weight      = FUR_Weight)
       
    f_predict = Class_data.get_Random_Forest().predict_proba(data_row.reshape(1, -1))

    e_predict = URM_Exp.exp_predict_proba(data_row.reshape(1, -1))
    e_predict = np.array(e_predict).ravel()
    
    acq_data = URM_Exp.get_acq_data()
    
#    X, y = URM_Exp.get_surrogate_data()

#    e_predict = sur_GP.predict(X, y)
#    e_predict = sur_GP.predict(URM_Exp.get_surrogate_data())

#    print(f_predict, e_predict, acq_data.get_fe_x0())
    
    scores = URM_Exp.permutation_importance()
    
    consistancy = URM_Exp.Y_Consistancy(N_points = N_consistancy, std_bound = consist_bound)
    
    URM_Exp.del_1_rel()
    
    del_1_var = URM_Exp.get_del_1_variance()
    
    Class_Stats.Add_Sample(sample_scores = scores,
                         outcome       = y_test[test_instance],
                         X_row         = data_row.reshape(1, -1),
                         f_prediction  = f_predict,
                         e_prediction  = e_predict.ravel(),
                         feopt         = acq_data.get_fe_x0(),
                         consistancy   = consistancy,
                         del_1_var     = del_1_var)
    
    
    ############################################################################################
    
    URM_Exp_2.explain(X_init      = data_row.reshape(1, -1),
                      Dimension   = 'Multi D',
                      kernel_type = "Matern_3",
                      max_iter    = 20,
                      alpha       = "FUR_W",
                      #jitter=5,
                      interval    = 1,
                      weight      = FUR_Weight)
       
    f_predict = Class_data.get_L_Regression().predict_proba(data_row.reshape(1, -1))

    e_predict = URM_Exp_2.exp_predict_proba(data_row.reshape(1, -1))
    e_predict = np.array(e_predict).ravel()
    
    scores = URM_Exp_2.permutation_importance()
    
    Class_Stats_2.Add_Sample(sample_scores = scores,
                           outcome       = y_test[test_instance],
                           X_row         = data_row.reshape(1, -1),
                           f_prediction  = f_predict,
                           e_prediction  = e_predict.ravel(),
                           feopt         = None)
    


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
print(f_predict)
print(e_predict)


In [ ]:
Class_Stats.Feature_Counts(max_features=20, scaled=True, threshold=0.05)


In [ ]:
Class_Stats.Frequency_Plot()


In [ ]:
Class_Stats.Class_Fidelity()

In [ ]:
Class_Stats.Jaccard_Values()

In [ ]:
Class_Stats.Box_Plot()


In [ ]:
Class_Stats.View_Explanation(instance = 0)

In [ ]:
scores = URM_Exp.permutation_importance()
    
print(scores)

In [ ]:
scores_k = URM_Exp.KL_imp()

print(scores_k)

In [ ]:
scores_v = URM_Exp.Var_imp()

print(scores_v)

In [ ]:
scores_d1 = URM_Exp.del_1_rel()

print(scores_d1)

In [ ]:
scores_d2 = URM_Exp.del_2_rel()

print(scores_d2)

In [ ]:
lin_scores = URM_Exp.Lin_scores()

print(lin_scores)

In [ ]:
URM_Exp.plot_scores(Title)

In [ ]:
print(URM_Exp.get_exp_L())


In [ ]:
acq_data.Create_BB_plot()

In [ ]:
acq_data.plot_point(0)

In [ ]:
acq_data.plot_point(5)

In [ ]:
acq_data.plot_point(10)

In [ ]:
acq_data.plot_point(15)

In [ ]:
acq_data.plot_point(19)

In [ ]:
acq_data.plot_all()

In [ ]:
acq_data.plot_t1_t2(0)

In [ ]:
acq_data.plot_t1_t2(1)

In [ ]:
acq_data.plot_t1_t2(2)

In [ ]:
acq_data.plot_t1_t2(3)

In [ ]:
acq_data.plot_t1_t2(4)

In [ ]:
acq_data.plot_t1_t2(5)

In [ ]:
acq_data.plot_t1_t2(6)

In [ ]:
acq_data.plot_t1_t2(7)

In [ ]:
acq_data.plot_t1_t2(8)

In [ ]:
acq_data.plot_t1_t2(9)

In [ ]:
acq_data.plot_t1_t2(10)

In [ ]:
acq_data.plot_t1_t2(11)

In [ ]:
acq_data.plot_t1_t2(12)

In [ ]:
acq_data.plot_t1_t2(13)

In [ ]:
acq_data.plot_t1_t2(14)

In [ ]:
acq_data.plot_t1_t2(15)

In [ ]:
acq_data.plot_t1_t2(16)

In [ ]:
acq_data.plot_t1_t2(17)

In [ ]:
acq_data.plot_t1_t2(18)

In [ ]:
acq_data.plot_t1_t2(19)

In [ ]:
Class_Stats.Class_Fidelity()

print('Fidelity: ', Class_Stats.fidelity)

In [ ]:
Class_Stats.Jaccard_Values(top_k=4)

print('Jaccard: ', Class_Stats.jaccard_similarities)

In [ ]:
Class_Stats.Consistancy(std_bound = consist_bound, plot=True, title = Title)

In [ ]:
Class_Stats.Compare_Models (model_b = Class_Stats_2)

In [ ]:
Class_Stats.Calibration()

In [ ]:
Class_Stats.add_bb_del_1(Class_data.get_Random_Forest())

Class_Stats.delete_one()

In [ ]:
Class_Stats.Results_Summary(Title)